# LinearRegression from Scratch


# 구현할 것
- 공부시간과 성적간의 관계를 모델링한다.
    - **머신러닝 모델(모형)이란** 수집한 데이터를 기반으로 입력값(Feature)와 출력값(Target)간의 관계를 하나의 공식으로 정의한 함수이다. 그 공식을 찾는 과정을 **모델링**이라고 한다.
    - 이 예제에서는 공부한 시험시간으로 점수를 예측하는 모델을 정의한다.
    - 입력값과 출력값 간의 관계를 정의할 수있는 다양한 함수(공식)이 있다. 여기에서는 딥러닝과 관계가 있는 **Linear Regression** 을 사용해본다.

# 데이터 확인
- 입력데이터: 공부시간
- 출력데이터: 성적

|공부시간|점수|
|-|-|
|1|20|
|2|40|
|3|60|

우리가 수집한 공부시간과 점수 데이터를 바탕으로 둘 간의 관계를 식으로 정의 할 수 있으면 **내가 몇시간 공부하면 점수를 얼마 받을 수 있는지 예측할 수 있게 된다.**   
수집한 데이터를 기반으로 앞으로 예측할 수있는 모형을 만드는 것이 머신러닝 모델링이다.

  

## 학습(훈련) 데이터셋 만들기
- 모델을 학습시키기 위한 데이터셋을 구성한다.
- 입력데이터와 출력데이터을 각각 다른 행렬로 구성한다.
- 하나의 데이터 포인트의 입력/출력 값은 같은 index에 정의한다.

### 선형회귀 (Linear Regression)
- Feature들의 가중합을 이용해 Target을 추정한다.
- Feature에 곱해지는 가중치(weight)들은 각 Feature가 Target 얼마나 영향을 주는지 영향도가 된다.
    - 음수일 경우는 target값을 줄이고 양수일 경우는 target값을 늘린다.
    - 가중치가 0에 가까울 수록 target에 영향을 주지 않는 feature이고 0에서 멀수록 target에 많은 영향을 준다.
- 모델 학습과정에서 가장 적절한 Feature의 가중치를 찾아야 한다.
      

\begin{align}
&\large \hat{y} = W\cdot X + b\\
&\small \hat{y}: \text{모델추정값}\\
&\small W: \text{가중치}\\
&\small X: \text{Feature(입력값)}\\
&\small b: \text{bias(편향)}
\end{align}



## Train dataset 구성
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `공부시간` 1개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다.   
  이 문제에서 예측할 항목은 `시험점수` 한개이다.

In [14]:
import numpy as np
study_hours = [[1], [2], [3]] # 하나의 값으로 구성된 데이터를 2차원으로 구성되어 묶어서 표현함.
scores = [[20],[40],[60]] 

np.shape(study_hours), np.shape(scores)

((3, 1), (3, 1))

In [15]:
# tensor로 정의
import torch
X_train = torch.tensor(study_hours, dtype=torch.float32)
y_train = torch.tensor(scores, dtype=torch.float32)

In [16]:
print(X_train.shape, y_train.shape)
print(X_train.dtype, y_train.dtype)
X_train.type()

torch.Size([3, 1]) torch.Size([3, 1])
torch.float32 torch.float32


'torch.FloatTensor'

## 파라미터 (weight, bias) 정의

In [17]:
X_train

tensor([[1.],
        [2.],
        [3.]])

In [18]:
# torch.manual_seed(0), 시드값 
# y_pred = X_train * weight = bias
weight = torch.randn(1, 1, requires_grad=True)  # shape: (1:X의 feature개수, 1:출력값(예측)의 개수.)
#           정규 분포 안에서 random 한 값으로 잡은 것.
bias = torch.randn(1, requires_grad=True)

print(weight.shape, bias.shape)
print(weight, bias)

torch.Size([1, 1]) torch.Size([1])
tensor([[-0.1758]], requires_grad=True) tensor([0.4454], requires_grad=True)


In [19]:
# 추론 (w * x + b) , 행렬 곱
pred = X_train @ weight + bias
pred

tensor([[ 0.2696],
        [ 0.0939],
        [-0.0819]], grad_fn=<AddBackward0>)

In [20]:
y_train  # 정답

tensor([[20.],
        [40.],
        [60.]])

In [21]:
pred - y_train  # 뺀 값은 0 이 되어야 좋은 값임.

tensor([[-19.7304],
        [-39.9061],
        [-60.0819]], grad_fn=<SubBackward0>)

In [22]:
# 오차 계산 - MSE
loss = (y_train - pred) ** 2
loss

# loss 를 합하여 하나의 값으로 업데이트 처리해야함.  오차의 평균값을 확인하여

tensor([[ 389.2876],
        [1592.4994],
        [3609.8330]], grad_fn=<PowBackward0>)

In [23]:
# 오차의 평균값 계산
loss = torch.mean(y_train - pred) ** 2
loss


tensor(1592.4994, grad_fn=<PowBackward0>)

In [24]:
# weight, bias의 gradient계산
loss.backward()

In [25]:
weight.grad, bias.grad

(tensor([[-159.6245]]), tensor([-79.8123]))

In [26]:
# weight 변경 (update)
## new_weight = weight - weight.grad * learning_rate 
# ** learning_rate - hyper parameter
lr = 0.01
new_weight = weight.data - weight.grad
new_bias = bias.data - bias.grad * lr

In [27]:
print(new_weight.data, weight.data)
print(new_bias.data, bias.data)

tensor([[159.4488]]) tensor([[-0.1758]])
tensor([1.2435]) tensor([0.4454])


In [28]:
pred2 = X_train @ new_weight + new_bias
loss2 = torch.mean((y_train - pred2)**2)

In [29]:
print("이전 loss:", loss)
print("update후 loss:", loss2)  # gradient 가 0이 될때까지 계속 반복함

이전 loss: tensor(1592.4994, grad_fn=<PowBackward0>)
update후 loss: tensor(91442.9766)


### 모델링

In [30]:
weight = torch.randn(1, 1, requires_grad=True)  
bias = torch.randn(1, requires_grad=True)

In [31]:
#추론하는 함수 (예측 모델)
def linear_model(X):
    return X @ weight + bias

# 오차계산 함수 (MSE)
def loss_fn(pred, y):
    return torch.mean((pred-y)**2)

### 학습
1. 모델을 이용해 추정한다.
   - pred = model(input)
1. loss를 계산한다.
   - loss = loss_fn(pred, target)
1. 계산된 loss를 파라미터에 대해 미분하여 계산한 gradient 값을 각 파라미터에 저장한다.
   - loss.backward()
1. optimizer를 이용해 파라미터를 update한다.
   - optimizer.step()  
1. 파라미터의 gradient(미분값)을 0으로 초기화한다.
   - optimizer.zero_grad()
- 위의 단계를 반복한다.   

In [32]:
epochs = 1000 # 반복 횟수
lr = 0.001
for epoch in range(epochs):
    
    # 1. 추정
    pred = linear_model(X_train)

    # 2. loss 계산
    loss = loss_fn(pred, y_train)

    # 3. parameter 들의 gradient를 계산(loss에 대한 변화)
    loss.backward()

    # 4. parameter update
    weight.data = weight.data - weight.grad * lr    # .data:현재값, .grad: gradient 계산값.
    bias.data = bias.data - bias.grad * lr

    # 5. gradient 초기화 (반복됨으로 초기화하는 업데이트를 해야함)
    weight.grad = None
    bias.grad = None

    ## 로그 - loss를 출력
    if epoch % 100 == 0 or epoch == (epochs-1):
        print(f"[{epoch+1}/{epochs}] loss: {loss.item():05f}")

[1/1000] loss: 2070.001221
[101/1000] loss: 228.587204
[201/1000] loss: 30.485535
[301/1000] loss: 8.926572
[401/1000] loss: 6.345324
[501/1000] loss: 5.815186
[601/1000] loss: 5.517218
[701/1000] loss: 5.255522
[801/1000] loss: 5.008505
[901/1000] loss: 4.773336
[1000/1000] loss: 4.551437


In [33]:
weight.data, bias.data

(tensor([[17.5225]]), tensor([5.6312]))

In [34]:
new_X = torch.tensor([[5], [4], [2]], dtype=torch.float32)
linear_model(new_X)

tensor([[93.2438],
        [75.7213],
        [40.6763]], grad_fn=<AddBackward0>)

# 다중 입력, 다중 출력
- 다중입력: Feature가 여러개인 경우
- 다중출력: Output 결과가 여러개인 경우

다음 가상 데이터를 이용해 사과와 오렌지 수확량을 예측하는 선형회귀 모델을 정의한다.  
[참조](https://www.kaggle.com/code/aakashns/pytorch-basics-linear-regression-from-scratch)


|온도(F)|강수량(mm)|습도(%)|사과생산량(ton)|오렌지생산량|
|-|-|-|-:|-:|
|73|67|43|56|70|
|91|88|64|81|101|
|87|134|58|119|133|
|102|43|37|22|37|
|69|96|70|103|119|


2차원 행렬값 (5, 3)  --->   (5, 2)
```
사과수확량  = w11 * 온도 + w12 * 강수량 + w13 * 습도 + b1
오렌지수확량 = w21 * 온도 + w22 * 강수량 + w23 *습도 + b2
```

- `온도`, `강수량`, `습도` 값이 **사과**와, **오렌지 수확량**에 어느정도 영향을 주는지 가중치를 찾는다.
    - 모델은 사과의 수확량, 오렌지의 수확량 **두개의 예측결과를 출력**해야 한다.
    - 사과에 대해 예측하기 위한 weight 3개와 오렌지에 대해 예측하기 위한 weight 3개 이렇게 두 묶음, 총 6개의 weight를 정의하고 학습을 통해 가장 적당한 값을 찾는다.
        - `개별 과일를 예측하기 위한 weight들 @ feature들` 의 계산 결과를  **Node, Unit, Neuron** 이라고 한다.
        - 두 과일에 대한 Unit들을 묶어서 **Layer** 라고 한다.
- 목적은 우리가 수집한 train 데이터셋을 이용해 **정확한 예측을 위한 weight와 bias 들**을 찾는 것이다.

## Train Dataset
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `온도, 강수량, 습도` 세개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다. 이 문제에서 예측할 항목은 `사과수확량, 오렌지 수확량` 2개의 값이다.

In [35]:
#  input: 생산환경 (temp, rainfall, humidity) : (5, 3)
environs = [
    [73, 67, 43], 
    [91, 88, 64], 
    [87, 134, 58], 
    [102, 43, 37], 
    [69, 96, 70]
]

# Targets: 생산량 - (apples, oranges) - (5, 2)
apple_orange_output = [
    [56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]
]

In [36]:
# Dataset을 torch.Tensor로 생성.
X = torch.tensor(environs, dtype=torch.float32)
y = torch.tensor(apple_orange_output, dtype=torch.float32)
X.shape, y.shape

(torch.Size([5, 3]), torch.Size([5, 2]))

In [37]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

## weight와 bias
- weight: 각 feature들이 생산량에 영향을 주었는지의 가중치로 feature에 곱해줄 값.
    - 사과, 오렌지의 생산량을 구해야 하므로 가중치가 두개가 된다.
    - weight의 shape: `(3, 2)`   가중치 =2, feature = 3
- bias는 모든 feature들이 0일때 생산량이 얼마일지를 나타내는 값으로 feature와 weight간의 가중합 결과에 더해줄 값이다.
    - 사과, 오렌지의 생산량을 구하므로 bias가 두개가 된다.
    - bias의 shape: `(2, )`

### Linear Regression model
모델은 weights `w`와 inputs `x`의 내적(dot product)한 값에 bias `b`를 더하는 함수.

$$
\hspace{2.5cm} X \hspace{1.1cm} \cdot \hspace{1.2cm} W \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\cdot
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$


<center style="font-size:0.9em">
$w_{11},\,w_{12},\,w_{13}$: 사과 생산량 계산시 각 feature들(생산환경)에 곱할 가중치   <br>
$w_{21},\,w_{22},\,w_{23}$: 오렌지 생산량 계산시 각 feature들(생산환경)에 곱할 가중치    
</center>

<center>
<img src="figures/3_unit_layer.png">
</center>
# feature, parameter, bias

In [38]:
# weight/bias 를 정의 -> 초기값은 random 값을 이용해서 생성.
weight = torch.randn(3, 2, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

weight.size(), bias.size()
# weight : (3:input feature개수,  2: output  개수)
# bias :  (2:output 개수, )

(torch.Size([3, 2]), torch.Size([2]))

In [39]:
weight

tensor([[ 1.3812,  0.9509],
        [-0.2441,  0.2801],
        [ 0.1050, -0.9635]], requires_grad=True)

In [40]:
bias

tensor([ 0.8184, -0.0584], requires_grad=True)

In [41]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [42]:
# X @ parametet + bias = 결과

In [43]:
# 한번 학습(parameter 최적화)  # 예측값
##추론
pred = X @ weight + bias
pred

tensor([[ 89.8099,  46.6883],
        [111.7509,  49.4513],
        [ 94.3693,  64.3122],
        [135.0937,  73.3231],
        [ 80.0407,  24.9915]], grad_fn=<AddBackward0>)

In [44]:
y # 원래의 정답

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [45]:
## loss 계산(MSE)
loss = (pred - y) ** 2
loss

tensor([[ 1143.1067,   543.4376],
        [  945.6174,  2657.2678],
        [  606.6702,  4718.0093],
        [12790.1895,  1319.3663],
        [  527.1276,  8837.6064]], grad_fn=<PowBackward0>)

In [46]:
## loss 계산(MSE)
loss = torch.mean((pred - y) ** 2) # 전체 추론한 결과의 평균오차를 계산.
loss

tensor(3408.8398, grad_fn=<MeanBackward0>)

In [47]:
# 평균 (5개의 data를 예측해서 loss 를 줄이는? 평균 오차)

In [48]:
# loss 를 가지고 parameter(weight들, bias들)의 gradient 계산.
loss.backward()

In [49]:
weight.data, weight.grad

(tensor([[ 1.3812,  0.9509],
         [-0.2441,  0.2801],
         [ 0.1050, -0.9635]]),
 tensor([[ 2614.9905, -3030.0322],
         [  865.9539, -4553.0527],
         [  914.1242, -2704.4114]]))

In [50]:
bias.data, bias.grad

(tensor([ 0.8184, -0.0584]), tensor([ 26.0129, -40.2467]))

In [51]:
# parameter update  ( 1번의 업데이트만 됨)
lr = 0.00001
weight.data = weight.data - lr * weight.grad
bias.data = bias.data - lr * bias.grad

In [52]:
## 업데이트된 파라미터로 추정 -> loss 계산
pred2 = X @ weight + bias
loss2 = torch.mean((pred2 - y)**2)

In [53]:
print(loss.item(), loss2.item())  # 값이 0 에 가까워질수록 양쪽 값에 오차가 없다는 뜻.?

3408.83984375 2990.943603515625


##  모델링

In [54]:
weight = torch.randn(3, 2, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

## 모델 정의
def model(X):
    return X @ weight + bias

## loss 함수(MSE)
def loss_fn(pred, y):
    return torch.mean((pred - y)**2)  # 전체 오차의 평균.

In [55]:
# epoch : 전체 데이터를 한번 학습하는 것.

In [56]:
epochs = 5000  # 전체를 한번 학습하는건
lr = 0.00001  # 1e-5
for epoch in range(epochs):

    # 1. 추론
    pred = model(X)

    # 2. loss 계산  (5개의 데이터에 대한 예측 결과와 - 사과, 오렌지에 대한 예측 결과.)
    loss = loss_fn(pred, y)

    # 3. 파라미터 들의 gradient 계산
    loss.backward()

    # 4. 파라이터 업데이트
    weight.data = weight.data - lr * weight.grad
    bias.data = bias.data - lr * bias.grad 

    # 5. gradient 초기화
    weight.grad = None
    bias.grad = None


    ## 100 epoch, 마지막 epoch에서 loss를 출력 => 학습 과정 log 출력
    if epoch % 100 == 0 or epoch == (epoch-1):
        print(f"[{epoch+1}/{epochs}] - {loss.item():.5f}") 
        # 몇번째 epoch 를 돌리고 있는지 앞에 나타내줌. 예) 100/5000 5천번중에 100번째에폭.,.

[1/5000] - 7299.95605
[101/5000] - 36.23597
[201/5000] - 15.27038
[301/5000] - 8.57965
[401/5000] - 6.03221
[501/5000] - 4.76155
[601/5000] - 3.94388
[701/5000] - 3.33188
[801/5000] - 2.84275
[901/5000] - 2.44218
[1001/5000] - 2.11133
[1101/5000] - 1.83726
[1201/5000] - 1.61000
[1301/5000] - 1.42149
[1401/5000] - 1.26511
[1501/5000] - 1.13537
[1601/5000] - 1.02774
[1701/5000] - 0.93844
[1801/5000] - 0.86437
[1901/5000] - 0.80291
[2001/5000] - 0.75192
[2101/5000] - 0.70962
[2201/5000] - 0.67453
[2301/5000] - 0.64541
[2401/5000] - 0.62126
[2501/5000] - 0.60122
[2601/5000] - 0.58459
[2701/5000] - 0.57080
[2801/5000] - 0.55935
[2901/5000] - 0.54986
[3001/5000] - 0.54199
[3101/5000] - 0.53545
[3201/5000] - 0.53004
[3301/5000] - 0.52554
[3401/5000] - 0.52180
[3501/5000] - 0.51871
[3601/5000] - 0.51614
[3701/5000] - 0.51401
[3801/5000] - 0.51225
[3901/5000] - 0.51078
[4001/5000] - 0.50956
[4101/5000] - 0.50855
[4201/5000] - 0.50771
[4301/5000] - 0.50702
[4401/5000] - 0.50644
[4501/5000] - 0.5

In [57]:
# 새로운 데이터로 추론
print(f"{3:04d}")

0003


In [58]:
p = model(X)
p

tensor([[ 57.2016,  70.1415],
        [ 82.1945, 100.8015],
        [118.6634, 132.9260],
        [ 21.0702,  37.0194],
        [101.9499, 119.1705]], grad_fn=<AddBackward0>)

In [59]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [60]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [61]:
new_X = torch.tensor([[68, 82, 56]], dtype=torch.float32)
new_X = new_X.unsqueeze(dim=0)  # dummy 축 늘림(dim=축번호)
new_X.shape

torch.Size([1, 1, 3])

In [62]:
model(new_X)

tensor([[[80.8626, 95.4789]]], grad_fn=<AddBackward0>)

# pytorch built-in 모델을 사용해 Linear Regression 구현

In [63]:
inputs = torch.tensor(
    [[73, 67, 43], 
     [91, 88, 64], 
     [87, 134, 58], 
     [102, 43, 37], 
     [69, 96, 70]], dtype=torch.float32)

In [64]:
targets = torch.tensor(
    [[56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]], dtype=torch.float32)

## nn.Linear
Pytorch는 nn.Linear 클래스를 통해 Linear Regression 모델을 제공한다.  
nn.Linear에 입력 feature의 개수와 출력 값의 개수를 지정하면 random 값으로 초기화한 weight와 bias들을 생성해 모델을 구성한다.

In [65]:
# import torch

# torch.nn.Linear(input feature개수, output개수)  # weight, bias, X@weight + bias

## Optimizer와 Loss 함수 정의
- torch.optim 모듈에 다양한 Optimizer 클래스가 구현되있다.
- torch.nn 또는 torch.nn.functional 모듈에 다양한 Loss 함수가 제공된다. 이중 mse_loss() 를 사용한다.

In [66]:
# 선형회귀 모델을 정의. torch.nn.Linear 클래스
import torch
import torch.nn as nn

model = nn.Linear(3, 2)   # 3: input feature 개수,  2: output 개수
# def model(X):

In [67]:
inputs.shape

torch.Size([5, 3])

In [68]:
# loss 함수
loss_fn = torch.nn.functional.mse_loss   # 함수
# loss_fn = torch.nn.MSELoss()   # 클래스 -> 객체를 생성해야함.
# def loss_fn(pred, y):

In [69]:
# optimizer (torch.optim 모듈에 정의):   weight.data = weight.data - lr * weight.grad
optimizer = torch.optim.SGD(            # Stochastic Gradient Descent (SGD)
    model.parameters(),                 # 최적화 대상 파라미터들을 model에서 조회해서 전달.
    lr = 0.00001,             # learning Rate

)

In [70]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.5578, -0.0957,  0.1109],
         [ 0.3675,  0.0442, -0.2038]], requires_grad=True),
 Parameter containing:
 tensor([-0.1041,  0.5252], requires_grad=True)]

## Model Train

In [71]:
epochs = 5000

for epoch in range(epochs):

    # 추론
    pred = model(inputs)

    # loss 계산
    loss = loss_fn(pred, targets)  # torch.nn.functional.mse_loss(pred, targets)   #(모델추정값, 정답) => 파라미터 작성순서.

    # gradient 계산
    loss.backward()

    # 파라미터 업데이트 : optimizer.step()
    optimizer.step()

    # 파라미터 초기화     w.grad=None, b.grad=None
    optimizer.zero_grad()

    ##############################################여기까지 한번의 학습이 끝남.

    # 현재 eopch 학습 결과를 log로 출력
    if epoch % 100 == 0 or epoch == epochs-1:
         print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

[0001/5000] - 11454.5947265625
[0101/5000] - 162.26736450195312
[0201/5000] - 63.12005615234375
[0301/5000] - 32.36638259887695
[0401/5000] - 21.304096221923828
[0501/5000] - 16.18185043334961
[0601/5000] - 13.070210456848145
[0701/5000] - 10.80805778503418
[0801/5000] - 9.02091121673584
[0901/5000] - 7.563387393951416
[1001/5000] - 6.3612589836120605
[1101/5000] - 5.365915298461914
[1201/5000] - 4.540711402893066
[1301/5000] - 3.8562519550323486
[1401/5000] - 3.2884421348571777
[1501/5000] - 2.8174004554748535
[1601/5000] - 2.4266180992126465
[1701/5000] - 2.102412462234497
[1801/5000] - 1.8334251642227173
[1901/5000] - 1.6102832555770874
[2001/5000] - 1.4251471757888794
[2101/5000] - 1.2715672254562378
[2201/5000] - 1.1441470384597778
[2301/5000] - 1.0384310483932495
[2401/5000] - 0.9507253766059875
[2501/5000] - 0.8779692649841309
[2601/5000] - 0.8176106214523315
[2701/5000] - 0.7675238847732544
[2801/5000] - 0.7259787917137146
[2901/5000] - 0.6915132403373718
[3001/5000] - 0.662917

In [72]:
# 추론 => Gradient 계산을 할 필요가 없다. => grad_fn 을 만들 필요가 없다.
with torch.no_grad(): # => grad_fn 을 만들지 않음.  단, 학습에서는 사용해야함.
    pred = model(inputs)

In [73]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [74]:
pred

tensor([[ 57.1495,  70.4038],
        [ 82.2079, 100.5584],
        [118.7156, 133.0619],
        [ 21.0892,  37.0372],
        [101.8979, 119.0343]])

In [75]:
# 학습 로직을 함수로 구현
def train(inputs, targets, epochs, modal, loss_fn, optimizer):
     
     for epoch in range(epochs):
        epochs = 5000
        
        # 추론
        pred = model(inputs)

        # loss 계산
        loss = loss_fn(pred, targets)  # torch.nn.functional.mse_loss(pred, targets)   #(모델추정값, 정답) => 파라미터 작성순서.

        # gradient 계산
        loss.backward()

        # 파라미터 업데이트 : optimizer.step()
        optimizer.step()

        # 파라미터 초기화     w.grad=None, b.grad=None
        optimizer.zero_grad()

        ##############################################여기까지 한번의 학습이 끝남.

        # 현재 eopch 학습 결과를 log로 출력
        if epoch % 100 == 0 or epoch == epochs-1:
            print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

In [76]:
model = nn.Linear(3, 2)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001)

In [77]:
train(inputs, targets, 5000, model , nn.functional.mse_loss, optimizer)

[0001/5000] - 12045.296875
[0101/5000] - 7.3861517906188965
[0201/5000] - 1.5651946067810059
[0301/5000] - 0.673272967338562
[0401/5000] - 0.5365880727767944
[0501/5000] - 0.5156328678131104
[0601/5000] - 0.5124197006225586
[0701/5000] - 0.5119235515594482
[0801/5000] - 0.5118404030799866
[0901/5000] - 0.5118255615234375
[1001/5000] - 0.5118188261985779
[1101/5000] - 0.5118154287338257
[1201/5000] - 0.5118108987808228
[1301/5000] - 0.5118058323860168
[1401/5000] - 0.511800229549408
[1501/5000] - 0.5117964148521423
[1601/5000] - 0.5117897987365723
[1701/5000] - 0.5117881298065186
[1801/5000] - 0.511783242225647
[1901/5000] - 0.5117788314819336
[2001/5000] - 0.5117694139480591
[2101/5000] - 0.5117672681808472
[2201/5000] - 0.5117610692977905
[2301/5000] - 0.5117601752281189
[2401/5000] - 0.5117507576942444
[2501/5000] - 0.511749267578125
[2601/5000] - 0.5117443799972534
[2701/5000] - 0.5117385983467102
[2801/5000] - 0.5117336511611938
[2901/5000] - 0.5117284655570984
[3001/5000] - 0.5117